# Example workflow for creating a Collection, uploading Datasets, and revising

### Set relevant variables

In [ ]:
api_key_file_path = "path/to/api-key-file"

In [ ]:
collection_form_metadata = {
    "name": "Name",  # required
    "description": "A sample description",  # required
    "contact_name": "First Last",  # required
    "contact_email": "firstlast@email.com",  # required
    "doi": "10.1093/ajae/aaq063",
    "links": [
        {
            "link_name": "sample protocol link",
            "link_url": "https://www.sample.link",
            "link_type": "PROTOCOL",
        },
        {
            "link_name": "sample lab website",
            "link_url": "https://www.lab.website",
            "link_type": "LAB_WEBSITE",
        }
    ],
}

### Set url and access token env vars

In [ ]:
from src.utils.config import set_api_access_config
from src.utils.logger import set_log_level  # can set_log_level("ERROR") for less logging; default is "INFO"
from pprint import pprint

set_api_access_config(api_key_file_path)

### Create new private Collection

In [ ]:
from src.collection import create_collection
new_collection_id = create_collection(collection_form_metadata)

### Create new, empty Dataset

In [ ]:
from src.dataset import create_dataset
dataset_id = create_dataset(new_collection_id)

### Determine local datafile and Dataset id

In [ ]:
datafile_path = "/absolute/path/to-datafile.h5ad"

### Upload file to Collection

In [ ]:
from src.dataset import upload_local_datafile
upload_local_datafile(datafile_path, new_collection_id, dataset_id)

### Create a second Dataset and upload datafile

In [ ]:
second_dataset_id = create_dataset(new_collection_id)
upload_local_datafile(datafile_path, new_collection_id, second_dataset_id)

### Update Collection

In [ ]:
from src.collection import update_collection
update_collection(new_collection_id, collection_form_metadata)

### Retrieve the Collection metadata

In [ ]:
from src.collection import get_collection
collection_info = get_collection(new_collection_id)
pprint(collection_info)

### <font color='#ff8000'>_\*\*Use the Data Portal UI to publish the Collection before proceeding (easy access via link above)\*\*_</font>

### Create a revision of the Collection

In [ ]:
# After publish, the collection will now have a new permanent, canonical ID across versions.
# Use this ID for revisions.
collection_info = get_collection(new_collection_id)
canonical_collection_id = collection_info['id']

from src.collection import create_revision
revision_id = create_revision(canonical_collection_id)

### Retrieve the revision Collection metadata

In [ ]:
revision_info = get_collection(revision_id)
pprint(revision_info)

### Add a third Dataset (to the revision)

In [ ]:
third_dataset_id = create_dataset(revision_id)
upload_local_datafile(datafile_path, revision_id, third_dataset_id)

### Replace the second Dataset

In [ ]:
# Upload replacement datafile to the Dataset's id (same as in published collection) in the revision collection
upload_local_datafile(datafile_path, revision_id, second_dataset_id)

In [ ]:
from src.collection import delete_collection
delete_collection(revision_id)